In [1]:
import numpy as np
import video_sync as vs
import utm
import math
import random

In [2]:
def extract_gps_data(jsonfile):
    data = vs.object_init(jsonfile)
    lat = vs.extract_attribute(data,"latitude")
    lon = vs.extract_attribute(data,"longitude")
    acc = vs.extract_attribute(data,"accuracy")
    time = np.array(vs.extract_attribute(data,"timestamp"))/1000
    frame = vs.extract_attribute(data, "frame")

    data_final = np.column_stack((time,frame,lat,lon,acc))
    return data_final

def extract_slam_data(trajectory):
    f = open(trajectory, "r")
    runs = []
    data = []
    for i,line in enumerate(f):
        if line == "################################################## \n":
            if len(data) != 0:
                runs.append(np.array(data,dtype='float'))
                data = []
        else:
            l = line.split(" ")
            data.append(l[0:4])
    runs.append(np.array(data,dtype='float'))
    return runs

In [3]:
def time_sync(gps_data, slam_runs):
    t1 = gps_data[0][0]
    t2 = slam_runs[0][0][0]

    offset = t2 - t1
    for slam_data in slam_runs:
        slam_data[:,0] = slam_data[:,0] - offset

    return slam_runs


In [4]:
def sample_three(gps_data, slam_data):
    #Find all the reasonably accurate GPS points (between 0 and 5 meters)    
    start = np.searchsorted(gps_data[:,0],slam_data[0][0],'left')
    end = np.searchsorted(gps_data[:,0], slam_data[-1][0],'left')
    
    good_points = np.where(np.logical_and(gps_data[start:end,4] > -1, gps_data[start:end,4]<20))
    indices = np.array(random.sample(good_points[0],3)) + start
    corsp_slam = []
    for t in gps_data[indices][:,0]:
        idx = np.searchsorted(slam_data[:,0],t, 'left')
        corsp_slam.append(idx)

    return gps_data[indices][:,[0,2,3]], slam_data[corsp_slam]

In [27]:
def scale_and_combine(gps_data, slam_data):
    gps_samples,slam_samples =  sample_three(gps_data, slam_data)
    utm_samples = []
    for i in xrange(0,3):
        u = list(utm.from_latlon(gps_samples[i][1], gps_samples[i][2]))[:2]
        u.append(0)
        utm_samples.append(u)

    A = find_affine_transformation2(np.array(slam_samples[:,1:4]),np.array(utm_samples))
    slam_converted = np.zeros((len(slam_data),3))
    utm_conv = transform_pt2(slam_runs[2][:,[1,2,3]],A)
    
    for i,obv in enumerate(slam_data):
        slam_converted[i][0] = obv[0]
        ### One and Three are the two axis aligned to the plane.
        gps_conv = utm.to_latlon(utm_conv[i][0], utm_conv[i][1],18, 'T')
        print str(gps_conv[0]) + ","+ str(gps_conv[1])
        slam_converted[i][1:3] = gps_conv

    return slam_converted

In [6]:
def slam2gps(jsonfile, frametrajectory):   
    gps_data = extract_gps_data(jsonfile)
    slam_data = extract_slam_data(frametrajectory)
    slam_data = time_sync(slam_data, gps_data)

    location_data_final = scale_and_combine(gps_data, slam_data)

    return location_data_final

In [7]:
def recover_homogenous_affine_transformation(p, p_prime):
    '''
    Find the unique homogeneous affine transformation that
    maps a set of 3 points to another set of 3 points in 3D
    space:

        p_prime == np.dot(p, R) + t

    where `R` is an unknown rotation matrix, `t` is an unknown
    translation vector, and `p` and `p_prime` are the original
    and transformed set of points stored as row vectors:

        p       = np.array((p1,       p2,       p3))
        p_prime = np.array((p1_prime, p2_prime, p3_prime))

    '''

    # construct intermediate matrix
    Q       = p[1:]       - p[0]
    Q_prime = p_prime[1:] - p_prime[0]

    # calculate rotation matrix
    R = np.dot(np.linalg.inv(np.row_stack((Q, np.cross(*Q)))),
               np.row_stack((Q_prime, np.cross(*Q_prime))))

    # calculate translation vector
    t = p_prime[0] - np.dot(p[0], R)

    # calculate affine transformation matrix
    return np.column_stack((np.row_stack((R, t)),
                            (0, 0, 0, 1)))



def transform_pt(point, trans_mat):
    a  = np.array([point[0], point[1], point[2], 1])
    ap = np.dot(a, trans_mat)[:3]
    return [ap[0], ap[1], ap[2]]



In [8]:
def find_affine_transformation2(p,p_prime):
    pad = lambda x: np.hstack([x, np.ones((x.shape[0], 1))])
    unpad = lambda x: x[:,:-1]
    X = pad(p)
    Y = pad(p_prime)

    A, res, rank, s = np.linalg.lstsq(X,Y)

    return A

def transform_pt2(point,trans_mat):
    pad = lambda x: np.hstack([x, np.ones((x.shape[0], 1))])
    unpad = lambda x: x[:,:-1]

    transform = lambda x:unpad(np.dot(pad(x),trans_mat))

    return transform(point)

In [16]:
gps_data = extract_gps_data("/home/carmera/Documents/data/json/cds-1145ca086c7f4f26-20170608-0742.json")
slam_runs = extract_slam_data("/home/carmera/KeyFrame-test.txt")
slam_runs = time_sync(gps_data, slam_runs)

In [10]:
gps_samples,slam_samples =  sample_three(gps_data, slam_runs[2])
utm_samples = []
for i in xrange(0,3):
    u = list(utm.from_latlon(gps_samples[i][1], gps_samples[i][2]))[:2]
    u.append(0)
    utm_samples.append(u)
slam_samples = np.around(slam_samples,5)
A = find_affine_transformation2(np.array(slam_samples[:,1:4]),np.array(utm_samples)) 

In [45]:
slam_data = slam_runs[2]
start = np.searchsorted(gps_data[:,0],slam_data[0][0],'left')
end = np.searchsorted(gps_data[:,0], slam_data[-1][0],'left')

good_points = np.where(np.logical_and(gps_data[start:end,4] > -1, gps_data[start:end,4]<10))

In [65]:
indices

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42]])

In [74]:
corsp_slam = []
for t in gps_data[:,0][indices][0]:
    print t
    idx = np.searchsorted(slam_data[:,0],t, 'left')
    corsp_slam.append(idx)


1496922180.0
1496922182.0
1496922184.0
1496922186.0
1496922188.0
1496922190.0
1496922192.0
1496922194.0
1496922196.0
1496922198.0
1496922200.0
1496922202.0
1496922204.0
1496922206.0
1496922208.0
1496922210.0
1496922212.0
1496922214.0
1496922216.0
1496922218.0
1496922220.0
1496922222.0
1496922224.0
1496922226.0
1496922228.0
1496922230.0
1496922232.0
1496922234.0
1496922236.0
1496922238.0
1496922240.0
1496922242.0
1496922244.0
1496922246.0
1496922248.0
1496922250.0
1496922252.0
1496922254.0
1496922256.0
1496922258.0
1496922260.0
1496922262.0


array([  1.49692218e+09,   1.49692218e+09,   1.49692218e+09,
         1.49692218e+09,   1.49692219e+09,   1.49692219e+09,
         1.49692219e+09,   1.49692219e+09,   1.49692219e+09,
         1.49692220e+09,   1.49692220e+09,   1.49692220e+09,
         1.49692220e+09,   1.49692220e+09,   1.49692221e+09,
         1.49692221e+09,   1.49692221e+09,   1.49692221e+09,
         1.49692221e+09,   1.49692222e+09,   1.49692222e+09,
         1.49692222e+09,   1.49692222e+09,   1.49692222e+09,
         1.49692223e+09,   1.49692223e+09,   1.49692223e+09,
         1.49692223e+09,   1.49692223e+09,   1.49692224e+09,
         1.49692224e+09,   1.49692224e+09,   1.49692224e+09,
         1.49692224e+09,   1.49692225e+09,   1.49692225e+09,
         1.49692225e+09,   1.49692225e+09,   1.49692225e+09,
         1.49692226e+09,   1.49692226e+09,   1.49692226e+09,
         1.49692226e+09,   1.49692226e+09,   1.49692227e+09,
         1.49692227e+09,   1.49692227e+09,   1.49692227e+09,
         1.49692227e+09,

In [54]:
for x in slam_data:
    print x[0]

1496922179.27
1496922179.6
1496922180.0
1496922180.2
1496922180.6
1496922180.81
1496922182.07
1496922182.54
1496922182.94
1496922183.33
1496922183.87
1496922184.47
1496922185.07
1496922185.54
1496922186.13
1496922186.67
1496922187.14
1496922187.68
1496922188.01
1496922188.53
1496922188.87
1496922189.2
1496922189.6
1496922189.87
1496922190.14
1496922190.4
1496922190.74
1496922191.21
1496922191.67
1496922192.07
1496922192.47
1496922192.87
1496922193.47
1496922194.41
1496922195.4
1496922196.13
1496922197.0
1496922197.61
1496922198.34
1496922199.14
1496922199.54
1496922200.67
1496922211.4
1496922249.54
1496922250.13
1496922250.67
1496922251.33
1496922251.94
1496922252.67
1496922253.47
1496922254.2
1496922255.0
1496922255.81
1496922256.67
1496922257.47
1496922258.27
1496922259.0
1496922259.74
1496922260.27
1496922261.2
1496922261.73
1496922262.21
1496922262.53
1496922263.07
1496922263.47


In [63]:
gps_data[indices][0][0]

array([  1.49692218e+09,  -1.00000000e+00,   4.07191666e+01,
        -7.39435647e+01,   0.00000000e+00])